In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [21]:
def calculate_angle(a, b, c=None):
    a = np.array(a)
    b = np.array(b)
    if c is not None:
        c = np.array(c)
        
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)
        
        if angle > 180.0:
            angle = 360 - angle
            
        return angle
    else:
        radians = np.arctan2(b[1]-a[1], b[0]-a[0])
        direction = np.degrees(radians)
        return direction
        

In [24]:
def distanceCalculate(p1, p2):
    """p1 and p2 in format (x1,y1) and (x2,y2) tuples"""
    dis = ((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2) ** 0.5
    return dis

In [19]:
def distanceCalculate(p1, p2):
    """p1 and p2 in format (x1,y1) and (x2,y2) tuples"""
    dis = ((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2) ** 0.5
    return dis

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [9]:
def detection_body_part(landmarks, body_part_name):
    return [
        landmarks[mp_pose.PoseLandmark[body_part_name].value].x,
        landmarks[mp_pose.PoseLandmark[body_part_name].value].y,
        landmarks[mp_pose.PoseLandmark[body_part_name].value].visibility
    ]

In [70]:
# Initialize the webcam
pullup_video = '../videos/pull_up_1_person.mp4'
squat_video = '../videos/IMG_0292.mp4'
push_up_video = '../videos/push_up_1_person.mp4'
triceps_push_down = '../videos/triceps_pushdown.mp4'
cap = cv2.VideoCapture(triceps_push_down)
# cap = cv2.VideoCapture(0)

# Squat counter variables
counter = 0
stage = None  # "down" or "up"
exercise = 'triceps_pushdown'
direction = 1
# Initialize Pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    start = 0
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print("Ignoring empty camera frame.")
            break

        # Recolor the frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detect the pose
        results = pose.process(image)
        image_height, image_width, _ = image.shape
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            if exercise == 'squat':

                # Get coordinates for the left hip, knee, and ankle
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                # Calculate the angle
                angle = calculate_angle(hip, knee, ankle)

                # Squat logic
                if angle > 160:  # Threshold for standing
                    if stage == 'down':
                        counter += 1
                        print(counter)  # Print the squat count
                    stage = 'up'
                if angle < 100:  # Threshold for squat
                    stage = 'down'
            elif exercise == 'pushup':

                # Get coordinates for the left hip, knee, and ankle
                shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                # Calculate the angle
                angle = calculate_angle(shoulder, elbow, wrist)
                if angle > 160:  # Threshold for standing
                    if stage == 'down':
                        counter += 1
                        print(counter)  # Print the squat count
                    stage = 'up'
                if angle < 70:  # Threshold for squat
                    stage = 'down'
            elif exercise == 'pullup':
                shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                angle = calculate_angle(shoulder, elbow, wrist)
                if angle < 90:  # Threshold for "up" phase of a pull-up (body lifted)
                    if stage == 'down':
                        counter += 1
                        print(counter)  # Print the pull-up count
                    stage = 'up'
                elif angle > 160:  # Threshold for "down" phase (body lowered)
                    stage = 'down'
            elif exercise == 'triceps_pushdown':
                shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                angle = calculate_angle(shoulder, elbow, wrist)
                if angle > 135:
                    if stage == 'close':
                        counter+=1
                        print(counter)
                    stage = 'open'
                elif angle < 90:
                    stage = 'close'
                
        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.namedWindow('Squat Detection', cv2.WINDOW_NORMAL)
        cv2.imshow('Squat Detection', image, )

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

close
close
close
close
close
close
close
close
close
close
142.06412328728626
1
146.49314840467446
151.02438169919023
155.41683291027573
158.51023233865845
156.99742601340654
156.3303568879932
155.62546168238288
155.0820011530037
158.0943709735528
156.85217318887857
156.9786653146156
155.7238357683469
154.64116148981736
155.07663158351008
154.7376304605627
155.8307561185796
155.10873376607947
155.00906302262402
154.73163431011298
155.17499794305687
154.02447454997747
153.21771900906066
152.54706697462822
151.86275938116293
150.2173296902756
150.34847935031473
149.3450178710243
146.98863924574857
143.7455039552705
140.92376542592802
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
close
141.2946811191193